This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'XXXXXXXXXXXX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [143]:
# First, import the relevant modules
import requests 
import json

In [144]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

#calling quandl api using requests
r= requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2007-07-13&end_date=2007-07-14&api=API_KEY')
print(r.json())


{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2007-07-13', 'end_date': '2007-07-14', 'frequency': 'daily', 'data': [['2007-07-13', 16.2, 16.21, 15.75, 15.9, None, 98350.0, 1563907.0, None, None, None]], 'collapse': None, 'order': None}}


In [177]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())


{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2007-07-13', 'end_date': '2007-07-14', 'frequency': 'daily', 'data': [['2007-07-13', 16.2, 16.21, 15.75, 15.9, None, 98350.0, 1563907.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [178]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
# (keep in mind that the date format is YYYY-MM-DD)

year2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api=API_KEY&start_date=2017-01-01&end_date=2017-12-31&api=API_KEY')

#2. Convert the returned JSON object into a Python dictionary.

# grab column names from json dataset_data object
columns = year2017.json()['dataset_data']['column_names']
#grab data from json dataset_data object
OneYearResponse = year2017.json()['dataset_data']['data']

#list variables to convert json object to list of dicts
templist=[]
dictlist = []

for i in OneYearResponse:
    for x in i:
        #add each item to temporary list
        templist.append(x)
    #add column names and temporary list to list of dictionaries dictlist
    dictlist.append(dict(zip(columns, templist)))
    #clear temporary list to use again in next iteration of loop 
    templist[:] = []


In [181]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.

#Save items from the Open column in list of dictionaries dictlist
seq = [x['Open'] for x in dictlist]

#find the mim and max of opening prices excluding 'None' values
HighestOpenIndex = seq.index(max(x for x in seq if x is not None))
LowestOpenIndex = seq.index(min(x for x in seq if x is not None))

#print min and max
print('The highest opening price for 2017: ', dictlist[HighestOpenIndex]['Open'], ' On ', dictlist[HighestOpenIndex]['Date'])
print('The lowest opening price for 2017: ', dictlist[LowestOpenIndex]['Open'], ' On ', dictlist[LowestOpenIndex]['Date']) 

The highest opening price for 2017:  53.11  On  2017-12-14
The lowest opening price for 2017:  34.0  On  2017-01-24


In [180]:
#4.  What was the largest change in any one day (based on High and Low price)?

#compute difference of high and low columns in list of dictionaries and save to list
seq = [(x['High'] - x['Low']) for x in dictlist]

#return index of the highest difference
LargestChangeIndex = seq.index(max(seq))

#print results using index returned above to return highest value in seq and date item with highest change in dictlist
print('Largest change for 2017: ', round(seq[LargestChangeIndex], 2), ' on ', dictlist[LargestChangeIndex]['Date'])

Largest change for 2017:  2.81  on  2017-05-11


In [187]:
#5 What was the largest change between any two days (based on Closing Price)?

#store items from Close column in dictlist and put in new list
seq = [x['Close'] for x in dictlist]

#store change between closing days starting from day 1 
changeSeq = [x - seq[i-1] for i, x in enumerate(seq)][1:]

#grab index of the largest difference from above
changeSeqIndex = changeSeq.index(max(changeSeq))
#print results using index from above of largest change and the dates
print('Largest change between two days based on closing price : ', round(max(changeSeq), 2), ' between ', dictlist[changeSeqIndex + 1]['Date'], ' and ', dictlist[changeSeqIndex]['Date'])


Largest change between two days based on closing price :  2.56  between  2017-08-08  and  2017-08-09


In [189]:
#6 What was the average daily trading volume during this year?

#store items from Traded Volume column in dictlist in new list
seq = [x['Traded Volume'] for x in dictlist]

#calculate the average by grabbing the sum of all items and dividing by length of seq
avgTradingVolume = sum(seq)/len(seq)

#print the average Trading Volume
print('Average Trading Volume for 2017 : ', round(avgTradingVolume, 2))

Average Trading Volume for 2017 :  89124.34


In [190]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function 
#for calculating the median.)

#function to find median takes a list
def findMedian(list):
    #sort the sequence from question 6 with items in Traded Volume of dictlist
    sortedSeq = sorted(list)
    #divide the length of list and store in index variable
    index = len(seq) / 2
    #if length of list is even add the item in the list that correspondes to above index and next index value and divide by two to get median
    if (index % 2 == 0):
        #convert to int 
        index = int(index)
        med = (sortedSeq[index] + sortedSeq[index+1])/2
        return med
    #if length of list is odd then add .5 to round index up to get index of value in list that is median
    else:
        index = index + 0.5
        #convert to int
        index = int(index)
        return sortedSeq[index]

#call function to find median and pass in seq and print result
median2017 = findMedian(seq)
print('The Median value for Trading Volume in 2017 is : ', median2017)


The Median value for Trading Volume in 2017 is :  76600.0
